In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
#from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('cleaned_augmented_2.csv')
df

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,readmitted_<30,readmitted_>30,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,9,0,0,1,0,0,0,1,0
1,0,0,0,25,1,0,0,1,0,0,...,5,4,0,0,1,0,0,0,0,0
2,1,1,1,28,0,0,0,0,1,0,...,0,4,0,0,1,0,0,0,0,0
3,1,0,1,27,0,0,0,1,1,1,...,1,9,0,0,1,0,0,0,0,0
4,1,1,1,24,0,0,0,1,1,1,...,0,4,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,1,1,1,45,0,0,0,0,1,1,...,0,5,0,0,1,0,0,0,0,0
253676,1,1,1,18,0,0,0,0,0,0,...,0,7,1,0,0,0,0,0,0,1
253677,0,0,1,28,0,0,0,1,1,0,...,2,5,0,0,1,0,0,0,1,0
253678,1,0,1,23,0,0,0,0,1,1,...,1,9,0,0,1,0,0,0,0,0


In [3]:
X = df.iloc[:,0:-1]
scaler = MinMaxScaler()
X_sc = scaler.fit_transform(X)
X = pd.DataFrame(X_sc, columns=X.columns)
y = df.loc[:, 'Diabetes_binary']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 1)

In [4]:
acc_list = []
roc_auc_ovr_List = []
f1_micro_List = []

## Trial 1

In [8]:
start = datetime.now()

In [9]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [10]:
best_model.cv_results_

{'mean_fit_time': array([ 4.3806247 ,  3.38458867,  3.70996242,  3.31921885,  3.77723057,
         3.4483922 ,  5.84528682,  3.38862891, 34.05756052,  3.35281987,
         4.05180383,  3.38630817,  4.09154546,  3.69572284,  4.25456004,
         3.27415876,  4.17047069,  3.43583052,  0.65617652,  0.77796314,
         0.88159356,  0.87543197,  0.83753331,  0.85844407,  0.85071521,
         0.89971638,  0.90057087,  0.90221264,  3.44342806,  0.86133146]),
 'std_fit_time': array([0.624228  , 0.19975216, 0.21528794, 0.15484528, 0.38847248,
        0.34299458, 3.79087744, 0.20929828, 4.89284511, 0.21705612,
        0.1995044 , 0.1687092 , 0.45453564, 0.48707906, 0.38846292,
        0.18133617, 0.44029364, 0.2912666 , 0.06753481, 0.05253252,
        0.05335588, 0.04681559, 0.04356012, 0.02968904, 0.03619098,
        0.03893984, 0.04305104, 0.02873817, 0.27658806, 0.02076749]),
 'mean_score_time': array([0.04053006, 0.03519249, 0.03516369, 0.03276143, 0.03383169,
        0.03475118, 0.03562136

In [14]:
best_model.cv_results_['rank_test_accuracy']

array([30, 29, 27, 25,  1, 23, 14, 22, 15, 17,  2, 20,  2,  2,  2, 15,  2,
        2, 28, 26, 23, 21, 17,  2,  2,  2,  2,  2,  2, 17])

In [15]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [16]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860646
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868727
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870156
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871132
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871447
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871344
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871398
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871378
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871398
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871398


In [17]:
acc_list.append(results[0:1])

In [18]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'lbfgs'}

In [14]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols


results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.712024
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.699787
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.698351
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.695671
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.695219
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.694857
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.694780
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.694794
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.694844


In [19]:
roc_auc_ovr_List.append(results[18:19])

In [20]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [21]:
results = pd.DataFrame( best_model.cv_results_['params'] )


results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860646
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868727
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870156
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871132
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871447
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871344
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871398
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871378
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871398
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871398


In [22]:
f1_micro_List.append(results[4:5])

## Trial 2

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 2)

In [24]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [25]:
best_model.cv_results_

{'mean_fit_time': array([ 3.94026377,  3.59192417,  3.83788204,  3.56049833,  4.23880861,
         3.47576578,  4.48458536,  3.53805666, 29.34059157,  3.2486536 ,
         4.29721823,  3.1292469 ,  4.1443958 ,  3.15488994,  4.03799834,
         3.36253641,  4.07495077,  3.30090339,  0.60226915,  0.73706663,
         0.78173201,  0.97019107,  0.97807169,  0.97526503,  0.95188808,
         0.93307862,  0.81913836,  0.79770076,  3.19881263,  0.79571426]),
 'std_fit_time': array([0.26827714, 0.30940718, 0.29274109, 0.32732482, 0.46560334,
        0.34944967, 0.32014679, 0.36872093, 5.45043547, 0.30093704,
        0.37750744, 0.18407931, 0.28661883, 0.27272725, 0.36262261,
        0.30428092, 0.26755065, 0.17872438, 0.03168996, 0.05928897,
        0.10505277, 0.19150329, 0.12292587, 0.10034138, 0.1418659 ,
        0.15483996, 0.1052491 , 0.02105577, 0.17027599, 0.02218333]),
 'mean_score_time': array([0.03570385, 0.03520765, 0.03429677, 0.0354193 , 0.04046116,
        0.04029939, 0.03635092

In [26]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25, 24,  1, 21, 22, 22,  3,  7, 14, 14, 18, 18, 18,  3,
       17, 29, 25,  2, 16,  5,  7,  7,  7,  7,  7,  7,  5])

In [27]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [28]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860986
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868816
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870137
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871457
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871531
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871590
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871566
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871561
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871561
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871575


In [29]:
acc_list.append(results[0:1])

In [30]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [31]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.645233
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.617675
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.608103
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.602985
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.601318
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.600308
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.600089
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.600060
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.600148


In [32]:
roc_auc_ovr_List.append(results[1:2])

In [33]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [34]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860986
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868816
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870137
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871457
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871531
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871590
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871566
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871561
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871561
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871575


In [35]:
f1_micro_List.append(results[5:6])

## Trial 3

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 3)

In [37]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [38]:
best_model.cv_results_

{'mean_fit_time': array([ 3.83425913,  3.12883043,  3.64660909,  2.94897342,  3.66368117,
         3.23431444,  5.07676797,  3.21932609, 37.80682819,  3.36534173,
         3.71150627,  3.30982718,  4.00191321,  3.22306211,  3.91506689,
         3.03676381,  3.96254191,  3.27362013,  0.54895196,  0.72744513,
         0.832376  ,  0.8460145 ,  0.80935409,  0.82585714,  0.82696729,
         0.82797039,  0.82416337,  0.76630599,  3.26691189,  0.79947612]),
 'std_fit_time': array([0.2265648 , 0.25699164, 0.28061907, 0.21040669, 0.21291128,
        0.38442774, 0.30423673, 0.23872425, 2.67728356, 0.43572367,
        0.7265313 , 0.63427613, 0.32039875, 0.31571628, 0.29345214,
        0.34544884, 0.57141861, 0.39907014, 0.02205815, 0.03769575,
        0.01891501, 0.02288818, 0.02070885, 0.02636346, 0.03776218,
        0.03510202, 0.03717411, 0.05195164, 0.33159502, 0.02505812]),
 'mean_score_time': array([0.03374293, 0.02962801, 0.03065093, 0.03023028, 0.03465598,
        0.03465242, 0.03388805

In [39]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25,  2,  5,  1,  3,  8, 11, 11, 16,  9,  7, 16, 16, 11,
       24, 28, 26,  5,  4, 11, 16, 16, 16, 16, 16,  9, 11])

In [40]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [41]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860523
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868535
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870235
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871117
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871221
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871201
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871255
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871216
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871181
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871181


In [42]:
acc_list.append(results[0:1])

In [43]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [44]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.647027
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.622696
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.608629
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.603202
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.600730
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.600513
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.599923
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.599464
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.599423


In [45]:
roc_auc_ovr_List.append(results[1:2])

In [46]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [47]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860523
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868535
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870235
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871117
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871221
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871201
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871255
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871216
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871181
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871181


In [48]:
f1_micro_List.append(results[6:7])

## Trial 4

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 4)

In [50]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [51]:
best_model.cv_results_

{'mean_fit_time': array([ 3.55223584,  3.3466258 ,  3.53682878,  3.44312558,  3.8852608 ,
         3.31105127,  6.57836411,  3.29348133, 27.54237146,  3.04074023,
         3.97157073,  3.18827581,  4.07937329,  3.04370422,  4.08045483,
         3.40549903,  4.01614037,  3.1975332 ,  0.56378927,  0.69363408,
         0.79622443,  0.79962151,  0.75091057,  0.78887646,  0.79482977,
         0.80001218,  0.803865  ,  0.79795809,  3.3427321 ,  1.03634036]),
 'std_fit_time': array([0.39841963, 0.12036444, 0.26783182, 0.2446778 , 0.30099021,
        0.247392  , 3.39143162, 0.16844598, 4.43220908, 0.40044589,
        0.24751587, 0.28373086, 0.34523474, 0.31811443, 0.33807551,
        0.33130735, 0.27580621, 0.40465446, 0.01005809, 0.01490455,
        0.03299577, 0.0236808 , 0.06966027, 0.03324357, 0.02761755,
        0.02332827, 0.03478391, 0.02270426, 0.61610691, 0.20458591]),
 'mean_score_time': array([0.03174005, 0.03432422, 0.03246238, 0.03379924, 0.03449111,
        0.03593814, 0.03830118

In [52]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25,  1,  4,  2, 21, 20, 23, 23,  9,  5,  9,  5,  5, 16,
       16, 28, 25,  3, 22, 18,  9,  9,  9,  9,  9,  8, 18])

In [53]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [54]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860730
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868570
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870181
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871314
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871625
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871600
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871620
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871575
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871580
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871575


In [55]:
acc_list.append(results[0:1])

In [56]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [57]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.645268
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.621625
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.608900
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.605946
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.603438
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.602937
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.602293
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.602236
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.602184


In [58]:
roc_auc_ovr_List.append(results[1:2])

In [59]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [60]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860730
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868570
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870181
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871314
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871625
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871600
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871620
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871575
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871580
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871575


In [61]:
f1_micro_List.append(results[4:5])

## Trial 5

In [62]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 5)

In [63]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [64]:
best_model.cv_results_

{'mean_fit_time': array([ 4.03425508,  3.22669585,  3.69786623,  3.11010406,  3.89753263,
         3.53725085,  4.79045019,  3.1439003 , 31.78257127,  3.11349502,
         4.15666118,  3.24421229,  4.109043  ,  3.40787208,  4.20525122,
         3.26945317,  3.80524766,  3.2251725 ,  0.55379417,  0.70221095,
         0.86766813,  1.07786105,  1.03371642,  1.03363779,  0.99528058,
         1.00050569,  0.78458393,  0.78568416,  3.24737141,  0.7827508 ]),
 'std_fit_time': array([0.70988557, 0.23223846, 0.24466512, 0.14194532, 0.46966466,
        0.53947189, 0.93041966, 0.14054258, 3.89469035, 0.12701765,
        0.4312792 , 0.3912407 , 0.26346308, 0.65355908, 0.59364877,
        0.31637882, 0.50746043, 0.30350596, 0.01406839, 0.01865435,
        0.2275691 , 0.2562744 , 0.24497255, 0.23290228, 0.22911254,
        0.24870473, 0.0175439 , 0.01467573, 0.16127375, 0.02012865]),
 'mean_score_time': array([0.03907421, 0.03267682, 0.03337252, 0.03273404, 0.03198926,
        0.0386827 , 0.03509638

In [65]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25,  1, 19, 18,  2, 22, 15, 15, 15,  5, 21,  5,  5,  5,
        4, 29, 26, 19,  3, 22,  5,  5,  5,  5,  5,  5, 22])

In [66]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [67]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860365
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868259
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.869905
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.870718
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871157
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.870895
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.870895
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.870915
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.870891
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.870900


In [68]:
acc_list.append(results[0:1])

In [69]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [70]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.646898
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.621925
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.606324
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.604386
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.599508
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.598815
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.598749
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.598466
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.598501


In [71]:
roc_auc_ovr_List.append(results[1:2])

In [72]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [73]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860365
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868259
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.869905
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.870718
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871157
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.870895
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.870895
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.870915
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.870891
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.870900


In [74]:
f1_micro_List.append(results[4:5])

## Trial 6

In [75]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 6)

In [76]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [77]:
best_model.cv_results_

{'mean_fit_time': array([ 3.67989964,  3.27875111,  3.55268047,  2.95264535,  3.86669946,
         3.34934487,  5.50261407,  3.09169595, 25.48187156,  3.31307075,
         4.02801998,  3.49474475,  4.2413981 ,  3.49847949,  4.02874799,
         3.09594798,  4.06451027,  3.27255731,  0.55583339,  0.69760544,
         0.80819983,  0.81386056,  0.77267804,  0.80236039,  0.81025002,
         0.8160243 ,  0.80712585,  0.78305824,  3.32110829,  0.80755968]),
 'std_fit_time': array([0.43124783, 0.27007019, 0.17057934, 0.36289452, 0.34824934,
        0.31127547, 3.04326894, 0.43773101, 4.17106867, 0.22477836,
        0.26034424, 0.22546544, 0.43195382, 0.57127854, 0.24909874,
        0.389996  , 0.27200302, 0.39953608, 0.01460972, 0.02637907,
        0.0165787 , 0.01053   , 0.06818121, 0.019456  , 0.0190502 ,
        0.02510649, 0.01908092, 0.03701242, 0.25991028, 0.01693304]),
 'mean_score_time': array([0.03194473, 0.03191979, 0.03116677, 0.03028927, 0.03234177,
        0.03498728, 0.03194454

In [78]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25, 21, 23, 22, 17,  1, 11, 19,  2,  2, 11, 16, 15,  2,
       19, 29, 25, 23, 17, 11,  2,  2,  2,  2,  2,  2, 11])

In [79]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [80]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860533
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868555
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.869792
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.870915
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871093
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871088
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871088
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871117
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871132
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871127


In [81]:
acc_list.append(results[0:1])

In [82]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [83]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.649623
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.618508
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.607205
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.602817
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.600213
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.599592
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.599525
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.599601
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.599526


In [84]:
roc_auc_ovr_List.append(results[1:2])

In [85]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [86]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860533
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868555
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.869792
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.870915
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871093
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871088
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871088
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871117
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871132
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871127


In [87]:
f1_micro_List.append(results[8:9])

## Trial 7

In [88]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 7)

In [89]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [90]:
best_model.cv_results_

{'mean_fit_time': array([ 3.65122905,  3.13813226,  3.70288281,  3.23497024,  3.84335847,
         3.41534805,  7.09110096,  3.8016108 , 25.94321277,  3.18369076,
         3.80366139,  3.25497293,  3.98476553,  3.21794021,  4.18425422,
         3.0496244 ,  4.28200037,  3.02222776,  0.54379284,  0.72130213,
         0.82497725,  0.80664604,  0.82014892,  0.82756445,  0.78576009,
         0.84258883,  0.81800382,  0.820227  ,  3.14792671,  0.73430715]),
 'std_fit_time': array([0.29974356, 0.15869736, 0.30259376, 0.33021399, 0.28688332,
        0.29547621, 6.08328629, 0.41433938, 4.87869132, 0.17572368,
        0.39795355, 0.22587859, 0.41172631, 0.30335227, 0.27915499,
        0.4791843 , 0.5253305 , 0.39589353, 0.01264058, 0.05262125,
        0.02815419, 0.02349476, 0.01887277, 0.02571198, 0.04329607,
        0.01050822, 0.0323959 , 0.01319452, 0.26507134, 0.05994588]),
 'mean_score_time': array([0.03290174, 0.03449361, 0.0337595 , 0.03486841, 0.03458583,
        0.0343286 , 0.03178513

In [91]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 22, 26,  5, 21, 23, 18,  7, 19,  1, 19,  8,  1,  8,  8,
        8, 28, 25,  5, 23,  1,  8,  8,  8,  8,  8,  8,  1])

In [92]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [93]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.861282
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.869028
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870629
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871605
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871580
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871625
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871610
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871600
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871620
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871620


In [94]:
acc_list.append(results[0:1])

In [95]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [96]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.646246
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.620877
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.607281
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.601183
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.598961
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.598211
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.597849
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.597970
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.597948


In [97]:
roc_auc_ovr_List.append(results[1:2])

In [98]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 10.0,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [99]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.861282
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.869028
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870629
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871605
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871580
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871625
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871610
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871600
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871620
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871620


In [100]:
f1_micro_List.append(results[11:12])

## Trial 8

In [101]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 8)

In [102]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [103]:
best_model.cv_results_

{'mean_fit_time': array([ 3.75080531,  3.36446002,  3.75934651,  3.18616269,  3.68624063,
         2.87800508,  5.14114368,  3.51263108, 27.73780339,  3.27908576,
         4.19539206,  3.16987226,  4.18491242,  3.18566754,  3.85543263,
         3.21814952,  4.06310272,  3.31734016,  0.72311349,  0.71358266,
         0.78933129,  0.79110131,  0.78845212,  0.79713032,  0.79536765,
         0.78302424,  0.78115354,  0.73345118,  3.31885636,  0.81676023]),
 'std_fit_time': array([0.19897   , 0.1908906 , 0.37367901, 0.20733495, 0.34414813,
        0.3947086 , 2.13487735, 0.50157699, 3.01278875, 0.41782961,
        0.26061286, 0.36336336, 0.24514236, 0.25821956, 0.34534619,
        0.24312055, 0.52865924, 0.6649475 , 0.03690029, 0.09360731,
        0.02604871, 0.01714707, 0.02288138, 0.02115841, 0.02539391,
        0.0247659 , 0.02567092, 0.03481695, 0.22580454, 0.01551785]),
 'mean_score_time': array([0.03359044, 0.03230343, 0.03605268, 0.03095515, 0.03086517,
        0.03041914, 0.0337477 

In [104]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25,  3,  1,  4, 23, 19, 19,  6,  6,  6,  6, 19,  5,  6,
        6, 28, 25,  1, 23,  6,  6,  6,  6,  6,  6, 19,  6])

In [105]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [106]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860301
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868106
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.869772
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.870743
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.870984
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871038
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.870935
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.870895
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.870900
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.870900


In [107]:
acc_list.append(results[0:1])

In [108]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [109]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.644051
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.620415
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.606829
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.602776
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.600733
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.599130
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.598516
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.598502
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.598502


In [110]:
roc_auc_ovr_List.append(results[1:2])

In [111]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [112]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860301
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868106
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.869772
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.870743
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.870984
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871038
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.870935
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.870895
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.870900
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.870900


In [113]:
f1_micro_List.append(results[5:6])

## Trial 9

In [114]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 9)

In [115]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [116]:
best_model.cv_results_

{'mean_fit_time': array([ 3.81404195,  3.29863496,  3.67973931,  3.18386858,  3.79206805,
         3.12478194,  5.31404982,  3.3342998 , 22.93795497,  3.28737216,
         3.84688201,  3.21711311,  4.11449006,  3.47707298,  3.99158392,
         3.20086236,  4.08531544,  3.58315196,  0.65118871,  0.80257869,
         0.78553126,  0.78475246,  0.78589613,  0.81626387,  0.78615971,
         0.75021381,  0.78681819,  0.78860142,  3.05998812,  0.82441604]),
 'std_fit_time': array([0.42957878, 0.59670179, 0.30380133, 0.33204467, 0.24284309,
        0.18985491, 4.24008378, 0.26816328, 3.18096944, 0.41882781,
        0.22163342, 0.38832972, 0.70127485, 0.21001656, 0.66482569,
        0.4266609 , 0.38626548, 0.70893669, 0.16217133, 0.22771697,
        0.02723293, 0.01633426, 0.03721344, 0.01374991, 0.02477595,
        0.05236972, 0.02391608, 0.03823321, 0.37355285, 0.01752674]),
 'mean_score_time': array([0.03375831, 0.03261621, 0.03203528, 0.02993832, 0.03121328,
        0.0299391 , 0.03146269

In [117]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25,  5, 24, 22,  1, 11,  6,  6,  6,  6,  6,  4, 11,  3,
       11, 29, 26, 23,  2, 19, 11, 11, 11, 11, 11, 21, 19])

In [118]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [119]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860750
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868772
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870265
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871230
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871344
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871314
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871329
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871368
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871339
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871344


In [120]:
acc_list.append(results[0:1])

In [121]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [122]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.648606
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.621541
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.607411
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.602978
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.600472
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.599970
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.600001
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.599628
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.599647


In [123]:
roc_auc_ovr_List.append(results[1:2])

In [124]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [125]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.860750
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.868772
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870265
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871230
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871344
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871314
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871329
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871368
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871339
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871344


In [126]:
f1_micro_List.append(results[7:8])

## Trial 10

In [127]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 10)

In [128]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [129]:
best_model.cv_results_

{'mean_fit_time': array([ 3.76432095,  3.23556762,  3.63526437,  3.08584995,  3.86892366,
         3.27888174,  4.66941769,  3.28319571, 34.78959284,  3.47551346,
         4.13314774,  3.09002459,  4.04790206,  3.01634719,  3.87201457,
         3.09820516,  3.8712846 ,  3.39026322,  0.5421638 ,  0.68056023,
         0.79440646,  0.81485946,  0.7936141 ,  0.79658685,  0.78473294,
         0.78687603,  0.79767401,  0.7683157 ,  3.23616278,  0.80910137]),
 'std_fit_time': array([0.418395  , 0.26922886, 0.29112241, 0.35162338, 0.26947284,
        0.21283686, 0.45579279, 0.21791899, 5.27978142, 0.38222731,
        0.29493676, 0.31975845, 0.49960318, 0.35585867, 0.30115152,
        0.40955517, 0.39246951, 0.25466022, 0.01757247, 0.06563742,
        0.02777561, 0.02780585, 0.01368417, 0.02430184, 0.01780923,
        0.0292079 , 0.01963487, 0.05009041, 0.22080929, 0.01233622]),
 'mean_score_time': array([0.03180506, 0.03167865, 0.03197293, 0.03070526, 0.03140495,
        0.03394287, 0.03226011

In [130]:
best_model.cv_results_['rank_test_accuracy']

array([30, 29, 27, 25, 24,  1, 23,  7, 15, 15,  3, 11,  9,  3, 14,  9,  3,
        3, 28, 25,  2,  7, 11, 15, 15, 15, 15, 15, 22, 11])

In [131]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [132]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.861351
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.869062
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870442
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871482
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871649
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871699
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871669
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871684
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871674
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871674


In [133]:
acc_list.append(results[0:1])

In [134]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'lbfgs'}

In [135]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.643509
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.617162
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.605441
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.601130
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.598988
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.598086
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.598116
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.597957
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.597954


In [136]:
roc_auc_ovr_List.append(results[18:19])

In [137]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [138]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.861351
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.869062
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.870442
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.871482
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.871649
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.871699
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.871669
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.871684
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.871674
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.871674


In [139]:
f1_micro_List.append(results[5:6])

In [140]:
end = datetime.now()
time_taken = end - start
print('Time: ',time_taken) 

Time:  3:00:48.846202


## Results

In [148]:
accuracy = [x['score_acc'][0] for x in acc_list]
accuracy_c = [x['C'][0] for x in acc_list]
accuracy_penalty = [x['penalty'][0] for x in acc_list]
roc = [x['score_precision'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
roc_c = [x['C'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
roc_penalty = [x['penalty'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
f1 = [x['score_f1_micro'].reset_index(drop=True)[0] for x in f1_micro_List]
f1_c = [x['C'].reset_index(drop=True)[0] for x in f1_micro_List]
f1_penalty = [x['penalty'].reset_index(drop=True)[0] for x in f1_micro_List]

In [149]:
data = {'Accuracy': accuracy, 'Accuracy C': accuracy_c, 'Accuracy Penalty': accuracy_penalty,
        'Precision': roc, 'Precision C': roc_c, 'Precision Penalty': roc_penalty,
        'F1_micro':f1, 'F1_micro C': f1_c, 'F1_micro Penalty': f1_penalty}

In [192]:
trainingResults  = pd.DataFrame(data = data)
pd.options.display.max_colwidth = 100
trainingResults

,Accuracy,Accuracy C,Accuracy Penalty,Precision,Precision C,Precision Penalty,F1_micro,F1_micro C,F1_micro Penalty
0,0.860646,0.0001,l1,0.712024,0.0001,l2,0.871447,0.01,l1
1,0.860986,0.0001,l1,0.645233,0.0001,l2,0.871590,0.01,l2
2,0.860523,0.0001,l1,0.647027,0.0001,l2,0.871255,0.10,l1
3,0.860730,0.0001,l1,0.645268,0.0001,l2,0.871625,1.00,l2
4,0.860365,0.0001,l1,0.646898,0.0001,l2,0.871157,0.10,l1
5,0.860533,0.0001,l1,0.649623,0.0001,l2,0.871132,10000.00,l1
6,0.861282,0.0001,l1,0.646246,0.0001,l2,0.871625,0.10,l1
7,0.860301,0.0001,l1,0.644051,0.0001,l2,0.871038,1.00,l1
8,0.860750,0.0001,l1,0.648606,0.0001,l2,0.871368,0.10,l1
9,0.861351,0.0001,l1,0.643635,0.0001,l2,0.871699,0.10,l1


In [193]:
trainingResults.to_csv('LR_trainingResults.csv')

### Testing Model

In [262]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, shuffle=True, random_state = 11)

In [263]:
LR = LogisticRegression(penalty = 'l1', C = 0.0001, solver = 'saga').fit(X_train, Y_train)

In [264]:
from sklearn.metrics import f1_score, precision_score, confusion_matrix
acc = LR.score(X_test, Y_test)
predicted = LR.predict(X_test)
f1 = f1_score(Y_test, predicted)
precision = precision_score(Y_test, predicted)
cm = confusion_matrix(Y_test, predicted)
tn, fp, fn, tp = confusion_matrix(Y_test, predicted).ravel()

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [265]:
print("Accuracy: " + str(acc) + '\n'
      + "Precision: " + str(precision) + '\n'
      + "F1: " + str(f1) + '\n'
      + "True Negative " + str(tn) + '\n'
      + "False Positive " + str(fp) + '\n'
      + "False Negative " + str(fn) + '\n'
      + "True Positive " + str(tp) + '\n')

Accuracy: 0.8575173446862189
Precision: 0.0
F1: 0.0
True Negative 43507
False Positive 0
False Negative 7229
True Positive 0



In [266]:
cm

array([[43507,     0],
       [ 7229,     0]], dtype=int64)

# Increasing C due to 0's in precision and f1

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, shuffle=True, random_state = 11)

In [6]:
LR = LogisticRegression(penalty = 'l1', C = 0.01, solver = 'saga').fit(X_train, Y_train)

In [7]:
from sklearn.metrics import f1_score, precision_score, confusion_matrix
acc = LR.score(X_test, Y_test)
predicted = LR.predict(X_test)
f1 = f1_score(Y_test, predicted)
precision = precision_score(Y_test, predicted)
cm = confusion_matrix(Y_test, predicted)
tn, fp, fn, tp = confusion_matrix(Y_test, predicted).ravel()

In [10]:
print("Accuracy: " + str(acc) + '\n'
      + "Precision: " + str(precision) + '\n'
      + "F1: " + str(f1) + '\n'
      + "True Negative: " + str(tn) + '\n'
      + "False Positive: " + str(fp) + '\n'
      + "False Negative: " + str(fn) + '\n'
      + "True Positive: " + str(tp) + '\n')

Accuracy: 0.8665444654683065
Precision: 0.5978632478632478
F1: 0.29240254990072106
True Negative: 42566
False Positive: 941
False Negative: 5830
True Positive: 1399



In [9]:
cm

array([[42566,   941],
       [ 5830,  1399]], dtype=int64)